In [44]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import calendar

In [45]:
os.chdir('c:\\Users\\tyler\\OneDrive\\Documents\\Python\\NFL')
os.getcwd()

'c:\\Users\\tyler\\OneDrive\\Documents\\Python\\NFL'

# Import 2021 Picks

In [46]:
df = pd.read_csv('backend\\data\\predictions\\2021_nn.csv')
df['date'] = pd.to_datetime(df['date'])
df = df[df['date'] >= datetime(2021, 9, 1)]
df['hit'] = df['prediction'] == df['outcome']
df.dropna(inplace=True)

## Betting Analysis 

In [47]:
# Per Unit
per_unit = df.groupby(['risk_unit']).aggregate({'profit': 'sum', 'hit': ['sum', 'count']})
per_unit['perc'] = (per_unit[('hit', 'sum')] / per_unit[('hit', 'count')] * 100).apply(round)
per_unit

profit hit       perc
                 sum sum count     
risk_unit                          
0.25       -0.570977  11    17   65
0.50       -3.282564  10    23   43
1.50      -18.123875  15    42   36
2.00       -5.482560   8    16   50
3.00      -33.841304   2    18   11
3.50       18.375000   3    11   27

In [48]:
df[df['risk_unit'] >= 2]

,date,home,ml_h,away,ml_a,outcome,prediction,risk_unit,profit,week,season,hit
3,2021-10-03,Dallas Cowboys,-220,Carolina Panthers,180,Dallas Cowboys,Carolina Panthers,3.5,-3.500000,4,2021,False
5,2021-10-03,Green Bay Packers,-260,Pittsburgh Steelers,210,Green Bay Packers,Green Bay Packers,2.0,0.769231,4,2021,True
6,2021-10-03,Miami Dolphins,-140,Indianapolis Colts,120,Indianapolis Colts,Indianapolis Colts,2.0,2.400000,4,2021,True
7,2021-10-03,Minnesota Vikings,-105,Cleveland Browns,-115,Cleveland Browns,Cleveland Browns,3.0,2.608696,4,2021,True
12,2021-10-03,San Francisco 49ers,-135,Seattle Seahawks,115,Seattle Seahawks,San Francisco 49ers,2.0,-2.000000,4,2021,False
13,2021-10-04,Los Angeles Chargers,-170,Las Vegas Raiders,150,Los Angeles Chargers,Los Angeles Chargers,2.0,1.176471,4,2021,True
14,2021-10-07,Seattle Seahawks,280,Los Angeles Rams,-360,Los Angeles Rams,Seattle Seahawks,3.5,-3.500000,5,2021,False
16,2021-10-10,Atlanta Falcons,-610,New York Jets,460,Atlanta Falcons,New York Jets,3.0,-3.000000,5,2021,False
17,2021-10-10,Carolina Panthers,-145,Philadelphia Eagles,125,Philadelphia Eagles,Carolina Panthers,3.0,-3.000000,5,2021,False
26,2021-10-10,Pittsburgh Steelers,110,Denver Broncos,-130,Pittsburgh Steelers,Denver Broncos,3.0,-3.000000,5,2021,False


In [49]:
# Per Month
df['month'] = df['date'].dt.month
df['month'] = df['month'].apply(lambda x: calendar.month_abbr[x])
per_month = df.groupby(['month']).aggregate({'profit': 'sum', 'hit': ['sum', 'count']})
per_month['perc'] = (per_month[('hit', 'sum')] / per_month[('hit', 'count')] * 100).apply(round)
per_month.sort_values(by=[('profit', 'sum'), ], ascending=False)

profit hit       perc
             sum sum count     
month                          
Feb    -0.500000   0     1    0
Dec    -1.312916  15    32   47
Nov    -2.669445  12    33   36
Jan   -17.901682   6    20   30
Oct   -20.542237  16    41   39

In [50]:
# Per Week
per_week = df.groupby(['week']).aggregate({'profit': 'sum', 'hit': ['sum', 'count']})
per_week['perc'] = (per_week[('hit', 'sum')] / per_week[('hit', 'count')] * 100).apply(round)
per_week

profit hit       perc
            sum sum count     
week                          
4     -0.356367   6    10   60
5    -11.353571   3     9   33
6    -11.107046   3    10   30
7     -3.775253   2     6   33
8      6.050000   2     6   33
9     11.718750   2     8   25
10    -5.900000   2     7   29
11    -2.282313   6    12   50
12    -6.205882   2     6   33
13    -2.891405   2     3   67
14    -6.851716   5    10   50
15     1.809985   3     9   33
16     6.620220   5    10   50
17    -1.601254   3     5   60
18   -12.388889   1     8   12
19    -2.961538   1     4   25
20    -0.950000   1     3   33
22    -0.500000   0     1    0

In [51]:
# Per Team
per_team = df.groupby(['prediction']).aggregate({'profit': 'sum', 'hit': ['sum', 'count']})
per_team['perc'] = (per_team[('hit', 'sum')] / per_team[('hit', 'count')] * 100).apply(round)
per_team.sort_values(by=[('profit', 'sum'), ], ascending=False)

profit hit       perc
                                sum sum count     
prediction                                        
Cleveland Browns          17.608696   2     6   33
Houston Texans            12.875000   1     2   50
Tampa Bay Buccaneers      10.411139   4     5   80
Minnesota Vikings          2.319118   2     2  100
Indianapolis Colts         2.150000   1     2   50
Kansas City Chiefs         2.110051   4     5   80
Los Angeles Chargers       1.176471   1     1  100
Tennessee Titans           0.850000   2     4   50
Jacksonville Jaguars       0.550000   1     1  100
Green Bay Packers          0.059553   2     4   50
Buffalo Bills             -0.325000   2     5   40
Cincinnati Bengals        -0.397368   2     5   40
Pittsburgh Steelers       -0.500000   0     1    0
New York Jets             -1.296154   2     7   29
Dallas Cowboys            -1.394737   1     2   50
Atlanta Falcons           -1.500000   0     1    0
Las Vegas Raiders         -1.636364   1     3   33
San Francisco 49ers       -2.736275   4     7   57
Los Angeles Rams          -3.000000   0     2    0
Chicago Bears             -4.352941   1     3   33
Arizona Cardinals         -4.550000   1     5   20
Philadelphia Eagles       -4.669591   2     5   40
Washington Football Team  -5.638889   2     6   33
New England Patriots      -7.439024   2     7   29
Denver Broncos            -7.958333   1     5   20
Seattle Seahawks          -9.919355   2     8   25
Baltimore Ravens         -10.765927   3    10   30
New Orleans Saints       -11.456349   3     9   33
Carolina Panthers        -13.500000   0     4    0